In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix

train_df = pd.read_csv('training-password-data.csv')
test_df  = pd.read_csv('testing-password-data.csv')

X_train = train_df.drop(columns=['strength'])
y_train = train_df['strength'].astype(str)
X_test  = test_df.drop(columns=['strength'])
y_test  = test_df['strength'].astype(str)

X_train = pd.get_dummies(X_train, columns=['Type'])
X_test  = pd.get_dummies(X_test,  columns=['Type']).reindex(columns=X_train.columns, fill_value=0)

numeric_cols = ['letterChars','numberChars','specialChars','numCharacters']
scaler = StandardScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols]  = scaler.transform(X_test[numeric_cols])

le = LabelEncoder().fit(y_train)
y_train_enc = le.transform(y_train)
y_test_enc  = le.transform(y_test)
num_classes = len(le.classes_)

y_train_cat = to_categorical(y_train_enc, num_classes)
y_test_cat  = to_categorical(y_test_enc,  num_classes)

input_dim    = X_train.shape[1]
hidden_units = (input_dim + num_classes) // 2

model = Sequential([
    Dense(hidden_units, activation='relu',    input_shape=(input_dim,)),
    Dense(num_classes,  activation='softmax'),
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(
    X_train, y_train_cat,
    validation_split=0.1,
    epochs=100,
    batch_size=32,
    callbacks=[es],
    verbose=2
)

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3737/3737 - 9s - 3ms/step - accuracy: 0.9529 - loss: 0.1978 - val_accuracy: 0.9992 - val_loss: 0.0345
Epoch 2/100
3737/3737 - 8s - 2ms/step - accuracy: 0.9997 - loss: 0.0185 - val_accuracy: 0.9995 - val_loss: 0.0097
Epoch 3/100
3737/3737 - 13s - 3ms/step - accuracy: 0.9997 - loss: 0.0061 - val_accuracy: 0.9995 - val_loss: 0.0054
Epoch 4/100
3737/3737 - 8s - 2ms/step - accuracy: 0.9998 - loss: 0.0034 - val_accuracy: 0.9995 - val_loss: 0.0038
Epoch 5/100
3737/3737 - 10s - 3ms/step - accuracy: 0.9998 - loss: 0.0025 - val_accuracy: 0.9995 - val_loss: 0.0032
Epoch 6/100
3737/3737 - 10s - 3ms/step - accuracy: 0.9998 - loss: 0.0021 - val_accuracy: 0.9995 - val_loss: 0.0023
Epoch 7/100
3737/3737 - 9s - 3ms/step - accuracy: 0.9998 - loss: 0.0019 - val_accuracy: 0.9995 - val_loss: 0.0021
Epoch 8/100
3737/3737 - 10s - 3ms/step - accuracy: 0.9998 - loss: 0.0017 - val_accuracy: 0.9995 - val_loss: 0.0017
Epoch 9/100
3737/3737 - 10s - 3ms/step - accuracy: 0.9998 - loss: 0.0016 - val_accuracy: 0.9995 

In [2]:
loss, acc = model.evaluate(X_test, y_test_cat, verbose=0)
print(f"\nTest loss: {loss:.4f}, Test accuracy: {acc:.4f}")

y_pred_prob = model.predict(X_test)
y_pred_enc  = np.argmax(y_pred_prob, axis=1)
y_pred      = le.inverse_transform(y_pred_enc)

print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred, digits=4))

print("=== Confusion Matrix ===")
cm = confusion_matrix(y_test, y_pred, labels=le.classes_)
print(pd.DataFrame(cm,
                   index=[f"actual={c}" for c in le.classes_],
                   columns=[f"pred={c}" for c in le.classes_]))


Test loss: 0.0024, Test accuracy: 0.9996
16609/16609 ━━━━━━━━━━━━━━━━━━━━ 17s 1ms/step

=== Classification Report ===
              precision    recall  f1-score   support

           0     0.9985    0.9994    0.9989     71711
           1     0.9999    0.9996    0.9998    396208
           2     0.9995    0.9999    0.9997     63551

    accuracy                         0.9996    531470
   macro avg     0.9993    0.9996    0.9995    531470
weighted avg     0.9996    0.9996    0.9996    531470

=== Confusion Matrix ===
          pred=0  pred=1  pred=2
actual=0   71668      43       0
actual=1     108  396069      31
actual=2       0       7   63544
